In [1]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import psycopg2
import cv2
import os
from datetime import datetime
from collections import Counter
import shutil

In [2]:
# Replace these with your database credentials
db_config = {
    'dbname': 'data',
    'user': 'adrien',
    'password': 'adrien9583!',
    'host': 'localhost',
    'port': '5432'
}

In [3]:
def definition_of_tags(text):
    words = text.split()
    tags_and_words = {
        "règlement écrit": ["stationnement", "surface", "plancher", "zone", "occupations", "utilisations", "sol", "article"],
        "zonage": ["zone", "stationnement", "occupations", "utilisations"],
        "padd": ["developpement", "services", "secteurs", "protéger", "maitriser", "trame", "préserver", "améliorer", "ville-center", "reconquete", "zones", "équipements", "urbain", "padd","espaces","maitriser"],
        "Arrêté municipal/préfectoral/départemental/régional": ["vu","code","L.","R.","arrété","présent","article","définition","loi"]
    }

    stop_words = []
    with open("stop_words.txt", "r") as file:
        for line in file:
            word = line.strip()
            if word:
                stop_words.append(word)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    word_counts = Counter(filtered_words)
    top_words = word_counts.most_common(5)

    # Initialize a list and a set to store tagged words
    tagged_words = []
    unique_tagged_words = set()

    # Initialize dictionaries to store word counts and tag counts
    word_counts = {}
    tag_counts = {}
    tagged_words_by_tag = {}

    # Iterate through words and apply tags based on word lists
    for word in words:
        for tag, word_list in tags_and_words.items():
            if word.lower() in word_list:
                tagged_word = (word, tag)
                if tag not in tagged_words_by_tag:
                    tagged_words_by_tag[tag] = []
                tagged_words_by_tag[tag].append(tagged_word)
                word_counts[word] = word_counts.get(word, 0) + 1
                tag_counts[tag] = tag_counts.get(tag, 0) + 1

    # Calculate the total percentage for each tag and store in a dictionary
    tag_percentages = {}
    total_percentage = 0
    for tag, tagged_words in tagged_words_by_tag.items():
        total_count_for_tag = tag_counts[tag]
        tag_percentage = sum(
            word_counts[word] / total_count_for_tag * 100 for word, _ in tagged_words)
        tag_percentages[tag] = tag_percentage
        total_percentage += tag_percentage

    if total_percentage == 0:
        return []
    scaling_factor = 100 / total_percentage
    for tag in tag_percentages:
        tag_percentages[tag] *= scaling_factor
    results = []
    for tag, normalized_percentage in tag_percentages.items():
        results.append(tag)
        # print(f"Tag: {tag} ({normalized_percentage}%)")

    return top_words

In [4]:
def convert_pdf_to_image(document_name):
    # Convert PDF to images with reduced memory usage
    temp_image_dir = 'temp_images'
    os.makedirs(temp_image_dir, exist_ok=True)
    pages = convert_from_path(
        f'pdfs/{document_name}.pdf',
        dpi=300,
        thread_count=2,
        output_folder=temp_image_dir
    )
    return f"Extract successfully : {len(pages)} documents"

In [5]:
def extract_txt_from_pdf(document_name, source):
    conn = psycopg2.connect(**db_config)
    postgres_write = True
    
    ppm_files = [file for file in os.listdir("temp_images") if file.endswith(".ppm")]
    
    # Apply OCR to each page
    if postgres_write:
        cursor = conn.cursor()
        
    for i, ppm_file in enumerate(ppm_files, start=0):
        ppm_path = os.path.join("temp_images", ppm_file)
        extracted_text = pytesseract.image_to_string(ppm_path)
        if postgres_write:
            cursor.execute(
                """INSERT INTO pdf_text (document_name,
                    page_number,
                    text_content,
                    present_tags,
                    source,
                    upload_date,
                    user_reading
                ) VALUES (%s, %s, %s, %s, %s, %s, %s)""", (document_name, i + 1, extracted_text, ["undefined"], source, str(datetime.now()), False))
            conn.commit()
        else:
            with open(f'page_{i + 1}.txt', 'w', encoding='utf-8') as text_file:
                text_file.write(extracted_text)
                
        os.remove(ppm_path)

    if postgres_write:
        cursor.close()
        conn.close()

In [12]:
# Example data
document_name = ["ZONE-Udr"]
present_tags = ['règlement', 'zone']
source = 'https://www.forcalqueiret.fr/?page_id=222831'
city_name = 'Forcalqueiret'

os.makedirs(f"pdfs/used_pdf/{city_name}", exist_ok=True)
for name in document_name:
    # convert_pdf_to_image(name)
    extract_txt_from_pdf(name, source)
    # shutil.move(f"pdfs/{name}.pdf", f"pdfs/used_pdf/{city_name}/{name}.pdf")
    # shutil.move(f"pdfs/{name}.pdf:Zone.Identifier", f"pdfs/used_pdf/{city_name}/{name}.pdf:Zone.Identifier")

In [27]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
cursor.execute("SELECT id, document_name, page_number, text_content FROM pdf_text ;" )
datas = cursor.fetchall() 
cursor.close()
conn.close()

import pandas as pd
df = pd.DataFrame(datas)
df['tags'] = None

for index, row in df.iterrows():
    df.at[index, 'tags'] = definition_of_tags(df.at[index, 3])